## Work on Colab 

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import argparse
import fileinput
import shutil
import pandas as pd

%cd /content/
df = pd.read_csv('processed_data1.csv')
df.head()

train = df[(df['train_val_test']==1) | (df['train_val_test']==3)]
train['classes'].value_counts()

valid = df[df['train_val_test']==2]
valid['classes'].value_counts()

In [ ]:
!gdown --id '17rFdLEySkRdV9jOZaXsAH9uw0XcCcp4O'
!unzip -q PyTorch-YOLOv3.zip

%cd /content/PyTorch-YOLOv3/
!pip3 install -q poetry
!poetry -q install

In [ ]:
%cd /content/PyTorch-YOLOv3/data/
!gdown --id '1FExoROsfs4o8KpFWqFQ3fx44Upz1fblG'
!unzip -q images.zip

!gdown --id '1PPgGczvZ2vghXWunFuyPXQHxKGXSVN4C'
!unzip -q Enhace_ExDark.zip

!gdown --id '1TVxBCiRhGRMQEKQay9nZumyUoNPdFpGy'
!unzip -q labels.zip

%cd ..

In [ ]:
%cd /content/PyTorch-YOLOv3/weights/
!gdown --id '1ZaClZ5MjeLe_4C1lboxkx3Ecb9U8FDg3'

%cd ..

In [ ]:
img_dir = '/content/PyTorch-YOLOv3/data/custom/images/'
os.chdir('/content/PyTorch-YOLOv3/data/custom/')

train_file = 'train.txt'

with open(train_file,'w') as tf:
    for img in tqdm(train['path']):
        file = img.split('/')[-1]
        path = os.path.join(img_dir,file)
        if os.path.isfile(path):
            tf.write(path)
            tf.write('\n')
    tf.close()

In [ ]:
img_dir = '/content/PyTorch-YOLOv3/data/custom/images/'
os.chdir('/content/PyTorch-YOLOv3/data/custom/')

valid_file = 'valid.txt'

with open(valid_file,'w') as vf:
    for img in tqdm(valid['path']):
        file = img.split('/')[-1]
        path = os.path.join(img_dir,file)
        if os.path.isfile(path):
            vf.write(path)
            vf.write('\n')
    vf.close()

## Download pth file(weight of YOLOv3)

In [ ]:
%cd /content/PyTorch-YOLOv3/checkpoints/
!gdown --id '1NnJkzdIRWJmCTYFBb5eCvUhf3UioYK-r'
!gdown --id '1BhTMObdsC5SyTSY7PU8lhYABk4S58FiW'
!ls

## Training

In [ ]:
!poetry run yolo-train --model config/yolov3-custom.cfg --data config/custom.data -e 30 --pretrained_weights weights/darknet53.conv.74

# save pth file to google cloud

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil

# 指定要複製的檔案路徑
source_file_path = '/content/PyTorch-YOLOv3/checkpoints/yolov3_ckpt_30.pth'  # 替換成你要上傳的檔案路徑

# 指定目的地路徑
destination_folder = '/content/drive/My Drive/result/'  # 替換成你要儲存的目的地資料夾路徑

# 複製單一檔案到 Google Drive 中的指定目的地
shutil.copy(source_file_path, destination_folder)

## Validating

In [ ]:
!poetry run yolo-test --model config/yolov3-custom.cfg --data config/custom.data --weights checkpoints/yolov3_ckpt_30.pth

## Demo

In [ ]:
%cd /content/PyTorch-YOLOv3/
import numpy as np
from numpy import expand_dims
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import cv2
from pytorchyolo import detect, models

In [ ]:
%cd /content/PyTorch-YOLOv3/
custom_config = './config/yolov3-custom.cfg'
model_2_weights = './checkpoints/yolov3_ckpt_30.pth'
model_1_weights = './checkpoints/yolov3_ckpt_23.pth'

In [ ]:
import numpy as np
import pandas as pd
from numpy import expand_dims
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import cv2
from pytorchyolo import detect, models
import matplotlib.pyplot as plt
import os

def predict_objects(config,weights,image):
    
    '''this function takes inputs as model's config,weights file and input image and gives output image 
    with objects predicted and bounding box drawn on input image'''
    
    # define the labels
    labels = {0:'bicycle',1:'boat',2:'bottle',3:'bus',4:'car',5:'cat',6:'chair',7:'cup',8:'dog',9:'motorbike',10:'people',11:'table'}
    
    # Load the YOLO model
    model = models.load_model(config, weights)
    output_folder = '/content/PyTorch-YOLOv3/data/custom/output/'
    # Load the image as a numpy array
    img = cv2.imread(image)
    file_name = os.path.basename(image)
    # Convert OpenCV bgr to rgb
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Runs the YOLO model on the image 
    boxes = detect.detect_image(model, img)
        
    # load the image
    data = pyplot.imread(image)
    plt.imsave(output_folder+file_name, data)
    # plot the image
    pyplot.imshow(data)
    
    # get the context for drawing boxes
    ax = pyplot.gca()
    
    # plot each box
    for box in boxes:
        # get coordinates
        x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
        
        # calculate width and height of the box
        width, height = x2 - x1, y2 - y1
        
        # create the shape
        rect = Rectangle((x1, y1), width, height, fill=False, color='white')
        # draw the box
        ax.add_patch(rect)
        # draw text and score in top left corner
        label = "%s (%.3f)" % (labels[box[5]], box[4])
        print(label)
        pyplot.text(x1, y1, label, color='white')
        
    # show the plot
    pyplot.show()

In [ ]:
random_images = ['/content/PyTorch-YOLOv3/data/custom/images/2015_00308.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_00293.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_02239.JPEG'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_05213.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_05235.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_02770.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_04820.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_00281.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_04034.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_05270.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_03332.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_02137.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_00351.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_00966.jpg'
,'/content/PyTorch-YOLOv3/data/custom/images/2015_02251.JPG']

##Remider: using model_2_weights require changing "Enhace_Dark" filename into "images"

In [ ]:
%cd /content/PyTorch-YOLOv3/
for image in random_images:
    predict_objects(custom_config,model_1_weights,image)

In [ ]:
%cd /content/PyTorch-YOLOv3/
for image in random_images:
    predict_objects(custom_config,model_2_weights,image)

## Demo one file

In [ ]:
def get_boxes(config,weights,image):
    
    # Load the YOLO model
    model = models.load_model(config, weights)

    # Load the image as a numpy array
    img = cv2.imread(image)

    # Convert OpenCV bgr to rgb
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Runs the YOLO model on the image 
    boxes = detect.detect_image(model, img)
    
    # Output will be a numpy array in the following format:
    # [[x1, y1, x2, y2, confidence, class]]

    return boxes


# draw all results
def draw_boxes(filename,boxes):
    # load the image
    data = pyplot.imread(filename)
    # plot the image
    pyplot.imshow(data)
    # get the context for drawing boxes
    ax = pyplot.gca()
    # plot each box
    for box in boxes:
        # get coordinates
        x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
        
        # calculate width and height of the box
        width, height = x2 - x1, y2 - y1
        
        # create the shape
        rect = Rectangle((x1, y1), width, height, fill=False, color='white')
        # draw the box
        ax.add_patch(rect)
        # draw text and score in top left corner
        label = "%s (%.3f)" % (labels[box[5]], box[4])
        print(label)
        pyplot.text(x1, y1, label, color='white')
    # show the plot
    pyplot.show()
labels = {0:'bicycle',1:'boat',2:'bottle',3:'bus',4:'car',5:'cat',6:'chair',7:'cup',8:'dog',9:'motorbike',10:'people',11:'table'}

In [ ]:
image = "/content/bbtest.jpg"

boxes = get_boxes(custom_config,model_1_weights,image)
draw_boxes(image,boxes)